<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#2.-Training" data-toc-modified-id="2.-Training-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>2. Training</a></span><ul class="toc-item"><li><span><a href="#2-1.-Data-selection" data-toc-modified-id="2-1.-Data-selection-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>2-1. Data selection</a></span></li><li><span><a href="#2-2.-Training-Data-Preparation" data-toc-modified-id="2-2.-Training-Data-Preparation-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>2-2. Training Data Preparation</a></span></li><li><span><a href="#2-3.-Set-Training-Parameter" data-toc-modified-id="2-3.-Set-Training-Parameter-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>2-3. Set Training Parameter</a></span></li><li><span><a href="#2-4.-Training" data-toc-modified-id="2-4.-Training-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>2-4. Training</a></span></li><li><span><a href="#2-5.-Meta-Save" data-toc-modified-id="2-5.-Meta-Save-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>2-5. Meta Save</a></span></li></ul></li></ul></div>

In [1]:
import sys, os
import pandas as pd
import numpy as np
import setting
import pathSetting
sys.path.append("../../..")

from KETIToolDL.CLUSTTool.common import p1_integratedDataSaving as p1
from KETIToolDL.CLUSTTool.common import p2_dataSelection as p2
from KETIToolDL.CLUSTTool.common import p3_training as p3

import torch
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cuda is available.


c:\Users\korea\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Training 

## 2-1. Data selection

In [2]:
# 1. Train Data
# 1-1. 저장 Data 확인
DataMeta = p1.readJsonData(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())

In [3]:
dataList

['ML_testX_ActionPatternData_Split3',
 'ML_testy_ActionPatternData_Split3',
 'ML_trainX_ActionPatternData_Split3',
 'ML_trainy_ActionPatternData_Split3',
 'trainX_ActionPatternData_Split3',
 'trainy_ActionPatternData_SplitNone',
 'testX_ActionPatternData_Split3',
 'testy_ActionPatternData_SplitNone']

In [4]:
dataList[2], dataList[3]

('ML_trainX_ActionPatternData_Split3', 'ML_trainy_ActionPatternData_Split3')

In [5]:
# 1-2. Select Train Data
## dataX
dataName_X = dataList[2]
dataSaveMode_X = DataMeta[dataName_X]["integrationInfo"]["DataSaveMode"]
windows = DataMeta[dataName_X]["integrationInfo"]["dataInfo"]["windows"]

## datay
dataName_y = dataList[3]
dataSaveMode_y = DataMeta[dataName_y]["integrationInfo"]["DataSaveMode"]

In [6]:
# 1-3. Read Train Data
## CSV 로 Local 에 데이터 저장되어 있을 경우
dataFolderName = "data_integrated_result"
current = os.getcwd()
dataFolderPath = os.path.join(current, dataFolderName)

dataX = p2.getSavedIntegratedData(dataSaveMode_X, dataName_X, dataFolderPath)
datay = p2.getSavedIntegratedData(dataSaveMode_y, dataName_y, dataFolderPath)
integration_freq_sec = DataMeta[dataName_X]["integrationInfo"]["integration_freq_sec"]

In [7]:
print(dataX.shape)
print(datay.shape)

(7352, 32)
(7352, 1)


In [8]:
## MSH
# UCI HAR Data Read
# folderAddress = './data/'
# model_list = ["LSTM_cf","GRU_cf", "CNN_1D_cf","LSTM_FCNs_cf"]
# model_name = model_list[2]

# dataX, datay, _, _ = setting.getTrainDataFromFilesForClassification(folderAddress, model_name)

## 2-2. Training Data Preparation

In [9]:
# 2. Training Data Preparation
# 2-1.
featureListX= list(dataX.columns)
featureListy= list(datay.columns)
target_col = 'value'

# 2-2.
cleanTrainDataParam = 'NoClean'#  Classification, Regression과 같이 X, y가 분리된 경우에는 현재 고정해서 사용해야함

# 2-3.
scalerParam="scale"
splitRatio = 0.8
scaleMethod='minmax'
modeX = 'windows_split'

scalerRootPath_X = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
scalerRootPath_y = None

# split train data into train/valiation data
# normalization
trainvalX, X_scalerFilePath = p3.getScaledData(scalerParam, scalerRootPath_X, dataX, scaleMethod)

# train data를 랜덤으로 test_size=split_ratio에 대하여 train/validation set으로 분할
split_ratio = 0.2
train_x, val_x, train_y, val_y = train_test_split(trainvalX, datay, test_size=split_ratio, shuffle=True)



# 2-4. Scale & Split Train/Validation
#scalerRootPath_X = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
#scalerRootPath_y = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
#train_x, val_x, X_scalerFilePath = p3.getTrainValData(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam, scaleMethod, modeX, windows)
#train_y, val_y, y_scalerFilePath = p3.getTrainValData(datay, featureListy, scalerRootPath_y, splitRatio, "NoScale")

['concat_emb1', 'concat_emb2', 'concat_emb3', 'concat_emb4', 'concat_emb5', 'concat_emb6', 'concat_emb7', 'concat_emb8', 'concat_emb9', 'concat_emb10', 'concat_emb11', 'concat_emb12', 'concat_emb13', 'concat_emb14', 'concat_emb15', 'concat_emb16', 'concat_emb17', 'concat_emb18', 'concat_emb19', 'concat_emb20', 'concat_emb21', 'concat_emb22', 'concat_emb23', 'concat_emb24', 'concat_emb25', 'concat_emb26', 'concat_emb27', 'concat_emb28', 'concat_emb29', 'concat_emb30', 'concat_emb31', 'concat_emb32']
Make New scaler File


### 2-2.1 Data Preparation (기존 module 거의 안씀, 기존꺼는 (n_samples, dims) 식으로 요약 DL 기준) # 22/10/24 추가

In [10]:
# UCI HAR Data Read
# 원래 아래부분도 from KETIToolDL.CLUSTTool.common import p3_training as p3 등에 녹아있어야 하나, p1이 아직 완전한 형태가 아니어서 일단은 분리해서 사용
folderAddress = './data/'
model_list = ["SemiTime_cf", "SMATE_cf"]
model_method = model_list[1]
split_ratio = 0.2
scaler_x_path = './scaler/HAR_temp_minmax_scaler_x_semi.pkl'
train_x, train_y,test_x, test_y = setting.load_data(folderAddress, model_method)

if model_method == 'SemiTime_cf':
    # split labeled data/unlabeled data in train_x
    train_labeled_x, train_labeled_y, train_unlabeled_x = setting.labeled_unlabeled_split(train_x, train_y)
    train_labeled_x, valid_x, train_labeled_y, valid_y = train_test_split(train_labeled_x, train_labeled_y, test_size=split_ratio, shuffle=True)
    # normalization
    train_labeled_x, train_unlabeled_x, valid_x = setting.get_train_val_data([train_labeled_x, train_unlabeled_x, valid_x], scaler_x_path)
    train_labeled_x, train_labeled_y, train_unlabeled_x, valid_x, valid_y
    data_dict = {'train_labeled_x': train_labeled_x,
                 'train_labeled_y' : train_labeled_y,
                 'train_unlabeled_x': train_unlabeled_x,
                 'val_x': valid_x,
                 'val_y': valid_y}
elif model_method == 'SMATE_cf':
    train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=split_ratio, shuffle=True)
    train_x, valid_x = setting.get_train_val_data([train_x, valid_x], scaler_x_path)
    data_dict = {'train_x': train_x,
                 'train_y' : train_y,
                 'val_x': valid_x,
                 'val_y': valid_y}
else:
    print('Choose the model correctly')


(7352, 128, 9)
(7352,)
(2947, 128, 9)
(2947,)
Save MinMaxScaler in path: ./scaler/HAR_temp_minmax_scaler_x_semi.pkl


## 2-3. Set Training Parameter

In [11]:
# 3. Set Training Parameter
# 3-1.
model_list = ["SemiTime_cf","SMATE_cf"]
model_method = model_list[1]

n_epochs = 150 # 학습 epoch 횟수, int(default: 1000, 범위: 1 이상)
batch_size = 256  # batch 크기, int(default: 16, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)
num_classes = 6 # class 개수

trainParameter = setting.modelConfig[model_method]
trainParameter['device']  = device
trainParameter['num_classes'] = num_classes

modelTags =["action", "sensor", "classification", "pattern", dataName_X, model_method]
trainDataType = "timeseries"
modelPurpose = "classification"

trainDataInfo = DataMeta[dataName_X]['integrationInfo']

# 3-2. 모델을 저장할 파일 패스를 생성
from KETIPreDataTransformation.general_transformation.dataScaler import encodeHashStyle
trainParameter_encode =  encodeHashStyle(str(trainParameter))

# 현재는 HAR_raw 데이터를 사용하므로 원래는 변경해야 하나 module 구조상 그대로 둠 
# 엄밀히 하면 DataMeta에 scaler 등 모든 값을 수정해야 하나, 최종 module이나 data preparation 단계가 확정이 안되었으므로 그대로 둠
ModelName = dataName_X+"_"+model_method

trainDataPathList = [ModelName, dataName_X, trainParameter_encode]
modelFilePath = p3.getModelFilePath(trainDataPathList, model_method)


In [12]:
modelFilePath

['./Models\\SMATE_cf\\ML_trainX_ActionPatternData_Split3_SMATE_cf\\ML_trainX_ActionPatternData_Split3\\b3a81fee867334aeb15ada033b5ca3b4\\model.pt']

In [13]:
trainParameter

{'num_seq': 128,
 'input_size': 9,
 'num_classes': 6,
 'sup_ratio': 0.3,
 'p_ratio': 0.1,
 'd_prime_ratio': 1.0,
 'kernels': [7, 5, 3],
 'num_layers': 3,
 'hidden_dim': 128,
 'lr': 0.0001,
 'device': 'cuda'}

In [14]:
ModelName

'ML_trainX_ActionPatternData_Split3_SMATE_cf'

## 2-4. Training

In [15]:
# 4. Training
from KETIToolDL.TrainTool.Semi.trainer import ClassificationML as CML

cml = CML(model_method, trainParameter)
cml.processInputData(data_dict, batch_size, windows) #labeled, unlabeled 는 따로 처리 
# cml.processInputData(train_x, train_y, val_x, val_y, batch_size, windows) -> 나중에는 여기 넣도록 다시 수정 가능
model = cml.getModel()
best_model, timeElapsed = cml.trainModel(model, modelFilePath, n_epochs)

c:\Users\korea\anaconda3\envs\torch\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Start training model

Epoch 1/150
train Loss: 0.2564
val Loss: 0.2277

Epoch 10/150
train Loss: 0.0999
val Loss: 0.5009

Epoch 20/150
train Loss: 0.0410
val Loss: 0.5455

Epoch 30/150
train Loss: 0.0304
val Loss: 0.0322

Epoch 40/150
train Loss: 0.0287
val Loss: 0.0333

Epoch 50/150
train Loss: 0.0281
val Loss: 0.0411

Epoch 60/150
train Loss: 0.0278
val Loss: 0.1088

Epoch 70/150
train Loss: 0.0273
val Loss: 0.0534

Epoch 80/150
train Loss: 0.0268
val Loss: 0.0833

Epoch 90/150
train Loss: 0.0261
val Loss: 0.3997

Epoch 100/150
train Loss: 0.0256
val Loss: 0.4106

Epoch 110/150
train Loss: 0.0250
val Loss: 0.3055

Epoch 120/150
train Loss: 0.0245
val Loss: 0.2533

Epoch 130/150
train Loss: 0.0241
val Loss: 0.1634

Epoch 140/150
train Loss: 0.0237
val Loss: 0.0762

Epoch 150/150
train Loss: 0.0236
val Loss: 0.0448

Training complete in 13m 26s
Best val loss: 0.031111


## 2-5. Meta Save

In [16]:
# 5. Meta Save
# 5-1. model name 확인
ModelName

'ML_trainX_ActionPatternData_Split3_SMATE_cf'

In [17]:
# 5-2. update MetaFile
modelInfoMeta ={
    "author":"KETI",
    "modelName": ModelName,
    "modelVersion":0.1,
    "trainDataInfo":trainDataInfo,
    "featureList":featureListX,
    "target":featureListy,
    "trainDataType":trainDataType,
    "modelPurpose":modelPurpose,
    "model_method":model_method,
    "modelTags":modelTags,
    "cleanTrainDataParam":cleanTrainDataParam,
    "NaNProcessingParam":{},
    "trainDataName":[dataName_X,dataName_y],
    "trainParameter":cml.parameter,
    "scalerParam":scalerParam,
    "files":{
        "modelFile":{
            "fileName":"model.pth",
            "filePath":modelFilePath
        },
        "XScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":X_scalerFilePath       
        },
        "yScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":scalerRootPath_y       
        }
    },
    "trainParameter":trainParameter,
    "n_epochs":n_epochs,
    "timeElapsed" : timeElapsed
}
modelInfoMeta = p3.updateModelMetaData(ModelName, modelInfoMeta, pathSetting.trainModelMetaFilePath)

In [18]:
modelInfoMeta

{'author': 'KETI',
 'modelName': 'ML_trainX_ActionPatternData_Split3_SMATE_cf',
 'modelVersion': 0.1,
 'trainDataInfo': {'startTime': '2022-01-01',
  'endTime': '2022-01-01 08:10:04',
  'dataInfo': {'windows': 1,
   'dataSize': '235264',
   'timeElapsed': '0:00:52.012849'},
  'processParam': {'refine_param': {'removeDuplication': {'flag': False},
    'staticFrequency': {'flag': False, 'frequency': None}},
   'outlier_param': {'certainErrorToNaN': {'flag': False},
    'unCertainErrorToNaN': {'flag': False, 'param': {}}},
   'imputation_param': {'serialImputation': {'flag': False,
     'imputation_method': [],
     'totalNonNanRatio': 80}}},
  'integration_freq_sec': 4,
  'cleanParam': 'NoClean',
  'DataSaveMode': 'CSV'},
 'featureList': ['concat_emb1',
  'concat_emb2',
  'concat_emb3',
  'concat_emb4',
  'concat_emb5',
  'concat_emb6',
  'concat_emb7',
  'concat_emb8',
  'concat_emb9',
  'concat_emb10',
  'concat_emb11',
  'concat_emb12',
  'concat_emb13',
  'concat_emb14',
  'concat_em